In [19]:
import tensorflow as tf
import numpy as np
from tensorflow.python.platform import gfile
from tensorflow.python.framework import tensor_util
import tensorflow.contrib.graph_editor as ge
import t3f
import tensorflow.keras.backend as K

In [20]:
assert(tf.__version__ == '1.15.0')

In [3]:
!pip3 install deepspeech


In [3]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.1/deepspeech-0.6.1-models.tar.gz
!tar xvf deepspeech-0.6.1-models.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   620  100   620    0     0   1172      0 --:--:-- --:--:-- --:--:--  1172
100 1172M  100 1172M    0     0  4629k      0  0:04:19  0:04:19 --:--:-- 5597k      0  0:03:38  0:00:12  0:03:26 6456k8k      0  0:03:37  0:00:20  0:03:17 5362k1  0:03:31 3817k27k      0  0:04:13  0:01:02  0:03:11 4106k      0  0:04:07  0:01:14  0:02:53 5679k     0  4488k      0  0:04:27  0:01:53  0:02:34 5082k8k      0  0:04:25  0:02:03  0:02:22 5131k  4628k      0  0:04:19  0:03:22  0:00:57 4556k19  0:03:24  0:00:55 4280k
x deepspeech-0.6.1-models/
x deepspeech-0.6.1-models/lm.binary
x deepspeech-0.6.1-models/output_graph.pbmm
x deepspeech-0.6.1-models/output_graph.pb
x deepspeech-0.6.1-models/trie
x deepspeech-0.6.1-models/output_graph.tflite


In [5]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.1/audio-0.6.1.tar.gz
!tar xvf audio-0.6.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   608  100   608    0     0   1304      0 --:--:-- --:--:-- --:--:--  1304
100  192k  100  192k    0     0   114k      0  0:00:01  0:00:01 --:--:--     0 --:--:--  223k
x audio/
x audio/2830-3980-0043.wav
x audio/Attribution.txt
x audio/4507-16021-0012.wav
x audio/8455-210777-0068.wav
x audio/License.txt


In [21]:
!deepspeech --model deepspeech-0.6.1-models/output_graph.pb --audio audio/2830-3980-0043.wav

Loading model from file deepspeech-0.6.1-models/output_graph.pb
TensorFlow: v1.14.0-21-ge77504ac6b
DeepSpeech: v0.6.1-0-g3df20fe
2020-03-13 11:17:46.179777: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loaded model in 1.6s.
Running inference.
experience proofsless
Inference took 10.170s for 1.975s audio file.


In [22]:
tf.reset_default_graph()

## Weight extraction

In [23]:
def load_deepspeech_graph(graph_path):
#GRAPH_PB_PATH = '/content/deepspeech-0.6.1-models/output_graph.pb'
  gr = tf.Graph()
  wts = {}
  with tf.Session(graph=gr) as sess:
    print("load graph")
    with gfile.FastGFile(graph_path,'rb') as f:
        graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='',)
    graph_nodes=[n for n in graph_def.node]
    names = []
    for t in graph_nodes:
        names.append(t.name)
    print(names)
    for n in graph_nodes:
        if n.op=='Const':
          wts[n.name] = tensor_util.MakeNdarray(n.attr["value"].tensor)
  return wts, gr

In [24]:
def make_t3f_subgraph(graph,input_shape,W,bias):
  model = tf.keras.Sequential()
  input_layer = tf.keras.Input((input_shape))
  tt_layer = t3f.nn.KerasDense([8,8,4,8],[8,8,8,4],tt_rank=25,activation="relu")
  model.add(input_layer)
  model.add(tf.keras.layers.Reshape((-1,input_shape)))
  model.add(tt_layer)
  model.compile(loss="mse", optimizer='rmsprop')
  with tf.Session() as sess:
    cores = sess.run(W.tt_cores)
  params = list(cores)
  params.append(bias)
  model.set_weights(params)
  f_g = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])
  #subgraph = tf.Graph()
  with tf.Session(graph=graph) as sess:
    tf.import_graph_def(f_g, name='',)
  return graph

In [25]:
def replace_layer_with_t3f(gr,wts,layer_name):
  bias = wts[layer_name+"/bias"]
  weights = wts[layer_name+"/weights"]
  tt_weights = t3f.to_tt_matrix(weights,shape=[[8,8,4,8],[8,8,8,4]],max_tt_rank=25)
  tt_subgraph = make_t3f_subgraph(gr,2048,tt_weights,bias)
  ge.detach(gr.get_operation_by_name("MatMul_1"))
  ge.detach(gr.get_operation_by_name("input_1"))
  l = ge.make_list_of_op(tt_subgraph)
  print(l)

  prev_layer_out = gr.get_operation_by_name("Minimum")
  tt_layer_in = tt_subgraph.get_operation_by_name("reshape/Reshape")
  shape = tt_subgraph.get_operation_by_name("reshape/Reshape/shape")
  ge.connect([prev_layer_out,shape],tt_layer_in)
  
  print(l[0])

  tt_layer_out = tt_subgraph.get_operation_by_name("keras_dense/activation/Relu")
  minimun_y = gr.get_operation_by_name("Minimum_1/y")
  next_layer_in = gr.get_operation_by_name("Minimum_1")
  ge.connect([tt_layer_out,minimun_y],next_layer_in)

  return tt_subgraph

In [26]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [27]:
tf.reset_default_graph()
wts, gr = load_deepspeech_graph("deepspeech-0.6.1-models/output_graph.pb")

load graph
['input_samples', 'ExpandDims/dim', 'ExpandDims', 'AudioSpectrogram', 'Mfcc/sample_rate', 'Mfcc', 'Reshape/shape', 'Reshape', 'mfccs', 'input_node', 'input_lengths', 'previous_state_c', 'previous_state_h', 'transpose/perm', 'transpose', 'Reshape_1/shape', 'Reshape_1', 'layer_1/bias', 'layer_1/bias/read', 'layer_1/weights', 'layer_1/weights/read', 'MatMul', 'BiasAdd', 'Relu', 'Minimum/y', 'Minimum', 'layer_2/bias', 'layer_2/bias/read', 'layer_2/weights', 'layer_2/weights/read', 'MatMul_1', 'BiasAdd_1', 'Relu_1', 'Minimum_1/y', 'Minimum_1', 'layer_3/bias', 'layer_3/bias/read', 'layer_3/weights', 'layer_3/weights/read', 'MatMul_2', 'BiasAdd_2', 'Relu_2', 'Minimum_2/y', 'Minimum_2', 'Reshape_2/shape', 'Reshape_2', 'cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/kernel', 'cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/kernel/read', 'cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/bias', 'cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_

In [28]:
gr_new = replace_layer_with_t3f(gr,wts,"layer_2")

INFO:tensorflow:Froze 5 variables.
INFO:tensorflow:Converted 5 variables to const ops.
[<tf.Operation 'input_samples' type=Placeholder>, <tf.Operation 'ExpandDims/dim' type=Const>, <tf.Operation 'ExpandDims' type=ExpandDims>, <tf.Operation 'AudioSpectrogram' type=AudioSpectrogram>, <tf.Operation 'Mfcc/sample_rate' type=Const>, <tf.Operation 'Mfcc' type=Mfcc>, <tf.Operation 'Reshape/shape' type=Const>, <tf.Operation 'Reshape' type=Reshape>, <tf.Operation 'mfccs' type=Identity>, <tf.Operation 'input_node' type=Placeholder>, <tf.Operation 'input_lengths' type=Placeholder>, <tf.Operation 'previous_state_c' type=Placeholder>, <tf.Operation 'previous_state_h' type=Placeholder>, <tf.Operation 'transpose/perm' type=Const>, <tf.Operation 'transpose' type=Transpose>, <tf.Operation 'Reshape_1/shape' type=Const>, <tf.Operation 'Reshape_1' type=Reshape>, <tf.Operation 'layer_1/bias' type=Const>, <tf.Operation 'layer_1/bias/read' type=Identity>, <tf.Operation 'layer_1/weights' type=Const>, <tf.Opera

In [29]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [30]:
def load_graph(frozen_graph_filename):
    with tf.io.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

def analyze_inputs_outputs(graph):
    ops = graph.get_operations()
    outputs_set = set(ops)
    inputs = []
    for op in ops:
        if len(op.inputs) == 0 and op.type != 'Const':
            inputs.append(op)
        else:
            for input_tensor in op.inputs:
                if input_tensor.op in outputs_set:
                    outputs_set.remove(input_tensor.op)
    outputs = list(outputs_set)
    return (inputs, outputs)

In [31]:
i,o = analyze_inputs_outputs(gr_new)

In [32]:
i

[<tf.Operation 'input_samples' type=Placeholder>,
 <tf.Operation 'input_node' type=Placeholder>,
 <tf.Operation 'input_lengths' type=Placeholder>,
 <tf.Operation 'previous_state_c' type=Placeholder>,
 <tf.Operation 'previous_state_h' type=Placeholder>,
 <tf.Operation 'input_1' type=Placeholder>,
 <tf.Operation 'geph__Minimum_0' type=Placeholder>,
 <tf.Operation 'layer_2/weights/geph__read_0' type=Placeholder>,
 <tf.Operation 'geph__MatMul_1_0' type=Placeholder>,
 <tf.Operation 'geph__input_1_0' type=Placeholder>]

In [33]:
for inp in i:
  print(inp.name)
  print(inp.get_attr("shape"))
  print("-------")

input_samples
dim {
  size: 512
}

-------
input_node
dim {
  size: 1
}
dim {
  size: 16
}
dim {
  size: 19
}
dim {
  size: 26
}

-------
input_lengths
dim {
  size: 1
}

-------
previous_state_c
dim {
  size: 1
}
dim {
  size: 2048
}

-------
previous_state_h
dim {
  size: 1
}
dim {
  size: 2048
}

-------
input_1
dim {
  size: -1
}
dim {
  size: 2048
}

-------
geph__Minimum_0
unknown_rank: true

-------
layer_2/weights/geph__read_0
unknown_rank: true

-------
geph__MatMul_1_0
dim {
  size: 16
}
dim {
  size: 2048
}

-------
geph__input_1_0
dim {
  size: -1
}
dim {
  size: 2048
}

-------


In [34]:
o

[<tf.Operation 'metadata_feature_win_len' type=Const>,
 <tf.Operation 'layer_2/weights/read' type=Identity>,
 <tf.Operation 'MatMul_1' type=MatMul>,
 <tf.Operation 'metadata_feature_win_step' type=Const>,
 <tf.Operation 'Relu_1' type=Relu>,
 <tf.Operation 'metadata_alphabet' type=Const>,
 <tf.Operation 'metadata_version' type=Const>,
 <tf.Operation 'mfccs' type=Identity>,
 <tf.Operation 'logits' type=Softmax>,
 <tf.Operation 'input_1' type=Placeholder>,
 <tf.Operation 'metadata_sample_rate' type=Const>,
 <tf.Operation 'new_state_c' type=Identity>,
 <tf.Operation 'new_state_h' type=Identity>]

In [0]:
i1 = gr.get_tensor_by_name('input_samples:0')
i2 = gr.get_tensor_by_name('input_node:0')
i3 = gr.get_tensor_by_name('input_lengths:0')
i4 = gr.get_tensor_by_name('previous_state_c:0')
i5 = gr.get_tensor_by_name('previous_state_h:0')
i6 = gr.get_tensor_by_name("input_1:0")
i7 = gr.get_tensor_by_name('geph__input_1_0:0')

t1 = np.ones(512,dtype=float) * 0.001
t2 = np.ones((1,16,19,26),dtype=float) * 0.021
t3 = [3]#np.array([3],dtype=np.int32)
t4 = np.zeros((1,2048),dtype=float) * 0.001
t5 = np.zeros((1,2048),dtype=float) * 0.001
t6 = np.zeros((1,2048),dtype=float)
t7 = np.zeros((1,2048),dtype=float)

out = gr.get_operation_by_name('logits')

In [0]:
with tf.Session(graph=gr) as sess:
 # sess.run(out,)
  x = gr.get_tensor_by_name("logits:0")
  res = x.eval(feed_dict={i1:t1,i2:t2,i3:t3,i4:t4,i5:t5,i6:t6,i7:t7})


In [58]:
res.shape

(16, 1, 29)

In [0]:

with tf.Session() as sess:
  with tf.gfile.GFile("x.pb", "wb") as f:
      f.write(gr.as_graph_def().SerializeToString())